### Package needed!

In [ ]:
_='''
pip install flask flask_cors
'''

### Create a database

In [ ]:
import os
import sqlite3 as sqlite
def create_db(db_file): 
    if not os.path.exists(db_file):
        print('Creating schema')
        conn = sqlite.connect(db_file)
        sql = "CREATE TABLE result (url TEXT, html_bin BLOB)"
        conn.execute(sql) # shortcut for conn.cursor().execute(sql)
        conn.commit() 
        conn.close()  
    else:
        print('Schema exists\n') 
create_db('w3school.db')
import pandas
pandas.read_sql_query("SELECT * FROM result", sqlite.connect('w3school.db'))

### Set up a web server

In [ ]:
from flask import Flask, request, make_response, jsonify 
from flask_cors import CORS  
 
app = Flask(__name__) 
CORS(app)

#### URL provider

In [ ]:
@app.route('/get_url') 
def get_url():
    return jsonify(url = [
                            'https://am-i-eligible.covid19vaccine.health.ny.gov',  
                            'https://www.w3schools.com/jsref/jsref_obj_json.asp',
                            'https://www.w3schools.com/jsref/jsref_obj_error.asp',
                            'https://www.w3schools.com/jsref/jsref_obj_array.asp'
                         ])

#### Database app

In [ ]:
@app.route('/database', methods = ['POST']) 
def database():  
    url = request.form['url'] 
    doc = request.form['doc'].encode() 
    conn = sqlite.connect('w3school.db')
    c = conn.cursor()   
    c.execute('''INSERT INTO result VALUES (?, ?)''', [url, sqlite.Binary(doc)]) 
    conn.commit() 
    conn.close()  
    print(url) 
    return 'Data has been inserted into the Database!' 

### Run both apps

In [ ]:
app.run('127.0.0.1', 9999)

### Inspect the result

In [ ]:
df = pandas.read_sql_query("SELECT * FROM result", sqlite.connect('w3school.db'))
df

In [ ]:
from IPython.core.display import HTML
display(HTML(df.loc[0,'html_bin'].decode()))

#### Extract titles from HTMLs

In [ ]:
import re
title = re.compile(b"<title(?![0-9A-Za-z]).*?>.*?</title>", re.DOTALL)

for row in df.itertuples():  
    m = re.search(title, row.html_bin) 
    print(m.group(0))